# VacationPy
----

Planning future vacations

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Importing API key
# To run this code, store google api as g_key
from api_keys import g_key

### Store Part I results into DataFrame
* Loading the csv exported in Part I to a DataFrame

In [2]:
#Reading and opening Weatherpy DataFrame
data_file = os.path.join("output_data","cities.csv")
weather_df= pd.read_csv(data_file)
weather_df.head()


,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,Ushuaia,AR,-54.80,-68.30,32.00,94,4,6.02,1592493596
1,Hobart,AU,-42.88,147.33,41.00,86,100,10.29,1592493512
2,Abhar,IR,36.15,49.22,79.93,30,2,8.19,1592493597
3,Rikitea,PF,-23.12,-134.97,76.08,83,87,21.14,1592493597
4,Barrow,US,71.29,-156.79,37.40,73,1,6.93,1592493302


### Humidity Heatmap

In [3]:
# Accessing maps with unique API key
gmaps.configure(api_key=g_key)

In [4]:
# Storing latitude and longitude in locations
locations = weather_df[["Latitude", "Longitude"]]

# Storing humidity in a variable that will be used as the "Weight" parameter for the heat map
humidity_values = weather_df["Humidity"].astype(float)

# Plotting Heatmap
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Defining maps parameters
fig = gmaps.figure(zoom_level=1.4, center= (51.4934,0.0098), layout=figure_layout)

# Creating heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_values, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)


# Adding the  layer
fig.add_layer(heat_layer)

# Displaying the  figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Filtering and adjusting the source dataframe to find desired weather conditions for future vacations

In [5]:
# Filtering ideal weather conditions, and creting a new dataframe
hotel_df= weather_df.loc[((weather_df["Max Temp"]>80) & (weather_df["Max Temp"]<=90)) & (weather_df["Wind Speed"]<10) \
                              & (weather_df["Humidity"]<50) & (weather_df["Cloudiness"]<5)].copy()
                                                                                                     
hotel_df

# Null values were removed in the previous analysis.

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
50,Ojinaga,MX,29.57,-104.42,82.40,42,1,3.36,1592493606
68,Canandaigua,US,42.87,-77.29,81.00,43,0,1.99,1592493534
434,Kumul,CN,42.80,93.45,88.39,16,0,8.66,1592493700
466,Pecos,US,31.42,-103.49,82.40,39,1,3.36,1592493610


### Hotel Map

In [6]:
# Adding empty columns to hotel_df
hotel_df["Hotel Name"]= ""
hotel_df["Lat"]= ""
hotel_df["Lng"]= ""
hotel_df

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name,Lat,Lng
50,Ojinaga,MX,29.57,-104.42,82.40,42,1,3.36,1592493606,,,
68,Canandaigua,US,42.87,-77.29,81.00,43,0,1.99,1592493534,,,
434,Kumul,CN,42.80,93.45,88.39,16,0,8.66,1592493700,,,
466,Pecos,US,31.42,-103.49,82.40,39,1,3.36,1592493610,,,


In [7]:
# Defining API url parameters
params = {
    "radius": 5000,
    "type":"lodging",
    "keyword":"hotel",
    "key": g_key
}

# Looping through each dataframe index and rows
for index, row in hotel_df.iterrows():
    # getting lat, lng from hotel_df
    lat = row["Latitude"]
    lng = row["Longitude"]
# changing location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Defining base Google Place API url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # making the request and getting the json response
    hotel_response = requests.get(base_url, params=params).json()
# Creating an exception to avoid keyerrors    
    try:
    
        print(f"Nice! we have found {hotel_response['results'][0]['name']} in {row['City']}")
        
        hotel_df.loc[index,'Hotel Name'] = hotel_response['results'][0]['name']
        hotel_df.loc[index,'Lat'] = hotel_response['results'][0]['geometry']['location']['lat']
        hotel_df.loc[index,'Lng'] = hotel_response['results'][0]['geometry']['location']['lng']
        
    except:
        #Printing an explanation message
        print(f"No Hotel Found in {row['City']}")
        # For those hotels that are not founded, th program must put np.NaN values over those cells
        hotel_df.loc[index,'Hotel Name'] = np.NaN
        hotel_df.loc[index,'Lat']= np.NaN
        hotel_df.loc[index,'Lng'] = np.NaN
        pass
    
# Dropping np.NaN values    
hotel_df.dropna(inplace=True)
        
    
# Organizing and printing hotel_df 

hotel_df= hotel_df[["City","Country","Lat","Lng","Hotel Name"]]
hotel_df

Nice! we have found Riata Inn in Ojinaga
Nice! we have found Holiday Inn Express Canandaigua - Finger Lakes in Canandaigua
Nice! we have found 7 Days Premium in Kumul
Nice! we have found Hampton Inn Pecos in Pecos


,City,Country,Lat,Lng,Hotel Name
50,Ojinaga,MX,29.5824,-104.375,Riata Inn
68,Canandaigua,US,42.8791,-77.2525,Holiday Inn Express Canandaigua - Finger Lakes
434,Kumul,CN,42.8278,93.5134,7 Days Premium
466,Pecos,US,31.3971,-103.518,Hampton Inn Pecos


In [8]:
# Defining info_box_template
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Storing info_box_content in hotel_info variable
# A list comprehension loop to store hotel info values
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Adding marker layer ontop of heat map
markers= gmaps.marker_layer(locations,info_box_content= hotel_info)
fig.add_layer(markers)

# Displaying figure

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…